In [12]:
import keras
from keras import backend as K
from keras.applications import resnet50
from keras.applications.imagenet_utils import decode_predictions
from keras.layers import Input
from keras.layers import Conv2D,Flatten,MaxPooling2D,Dense,Dropout,Activation
from keras.models import Model,Sequential
from keras.optimizers import Adam
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
import cv2
import numpy as np

In [13]:
def pretrained_resnet(inp):
    #we will be using pretrained ResNet50 on ImageNet
    #image_input = Input(shape=(224, 224, 3))
    resnet50_model = resnet50.ResNet50(input_tensor=inp, include_top=False, weights='imagenet')  
    resnet50_model.layers.pop()
    resnet50_model.layers.pop()
    resnet50_model.layers.pop()
    resnet50_model.layers.pop()
    resnet50_model.layers.pop()
    return resnet50_model

In [14]:
def new_img_size(width, height, img_min_side=600):
    if width <= height:
        k = float(img_min_side) / width
        resized_height = int(k * height)
        resized_width = img_min_side
    else:
        k = float(img_min_side) / height
        resized_width = int(k * width)
        resized_height = img_min_side

    return resized_width, resized_height

In [15]:
#import import_ipynb
#from Dataextract import data
#all_imgs, cls_count, cls_map  =data(r'C:\Users\ANSHUL\Downloads\VOCdevkit')


In [24]:
#w=all_imgs[0]['width']
#h=all_imgs[0]['height']
#r_w,r_h=new_img_size(w,h,img_min_side=600)

import tensorflow as tf
inp = Input(shape=(224,224,3))
layer_out = pretrained_resnet(inp)

for layer in layer_out.layers:
    layer.trainable = False
    
#print(model.layers[-1].output_shape)

x= Flatten(name='flatten')(layer_out.layers[-1].output)
print(tf.shape(x))
preds = Dense(50, activation = 'softmax')(x)

model = Model(inp, layer_out.layers[-1].output)
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

train_datagen = ImageDataGenerator(
        rescale=1./255,# set rescaling factor (applied before any other transformation)
        data_format=None )

train_generator = train_datagen.flow_from_directory(
        r'/home/ug2017/mec/17135096/faster_rcnn/Train_data',  # this is the target directory
        target_size=(224, 224),  # all images will be resized to 224x224
        batch_size=32)

model.fit_generator(
        train_generator,
        steps_per_epoch= (7535 // 32)+1,
        epochs=10)

Tensor("Shape_1:0", shape=(2,), dtype=int32)
Found 7535 images belonging to 50 classes.
Epoch 1/10


ValueError: Error when checking target: expected activation_636 to have 4 dimensions, but got array with shape (32, 50)

In [ ]:
model.summary()
model.layers[-1].output_shape